In [1]:
%rm -rf stonks/

In [2]:
## Retrieve csv source data from 'AnalyticsArmory' github folder
## Source data will be imported in /data/ subfolder
%%capture
!git clone https://github.com/IS737StockPicker/stonks.git

In [3]:
# import required packages
%matplotlib inline
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier #this is the neural network part
import matplotlib.pylab as plt
from sklearn.metrics import accuracy_score

In [4]:
# Special package from the class book to import regression summary statistics
!pip install -U dmba;
from dmba import regressionSummary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 13.2 MB/s eta 0:00:00
no display found. Using non-interactive Agg backend


In [52]:
# load the data
stocks_df = pd.read_csv('/content/stonks/master_data.csv')
stocks_df

,Date,XLE,XLF,XLU,XLI,XLK,XLV,XLY,IYR,AAPL,...,guardian_headline_median,guardian_body_median,Huff_headline_mean,Huff_body_mean,Huff_headline_median,Huff_body_median,NYT_headline_mean,NYT_headline_median,NYT_body_mean,NYT_body_median
0,2022-03-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,-0.00,-0.12,-0.06,0.00,0.00,-0.05,0.0,-0.03,0.00
1,2022-03-15,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,-0.15,-0.54,-0.44,-0.39,-0.46,-0.47,-0.09,0.0,0.02,0.00
2,2022-03-16,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.00,-0.55,0.10,-0.09,0.00,-0.13,-0.06,0.0,-0.00,0.00
3,2022-03-17,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.00,-0.85,-0.20,-0.14,-0.03,-0.03,-0.02,0.0,-0.05,0.00
4,2022-03-18,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.00,-0.31,-0.35,-0.30,-0.32,-0.22,-0.06,0.0,-0.05,-0.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,2022-09-07,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.00,0.83,-0.19,-0.19,0.00,-0.33,0.09,0.0,-0.07,0.00
121,2022-09-08,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.00,0.89,-0.25,-0.76,-0.48,-0.75,-0.04,0.0,-0.02,0.00
122,2022-09-09,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.00,0.95,-0.10,0.25,-0.10,0.25,-0.01,0.0,0.14,0.14
123,2022-09-12,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.00,0.10,-0.40,0.27,-0.42,0.38,-0.10,0.0,-0.06,-0.13


In [43]:
tickers = stocks_df.iloc[:,:15]
tickers = tickers.drop(columns=['Date'])
tickers

,XLE,XLF,XLU,XLI,XLK,XLV,XLY,IYR,AAPL,XLB,XLP,SPY,^DJI,NDX
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
121,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
122,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
123,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0


In [64]:
accuracy_scores={}
for ticker in tickers:
  #print(ticker)
  selected_var = [ticker,'guardian_headline_mean','guardian_body_mean','Huff_headline_mean','Huff_body_mean','NYT_headline_mean','NYT_body_mean']
  accuracy_df = stocks_df[selected_var]
  y_nonscaled = accuracy_df[[ticker]]
  X_nonscaled = accuracy_df.drop(columns=[ticker])
  scaleOutput = MinMaxScaler()
  scaleInput = MinMaxScaler()
  y = scaleOutput.fit_transform(y_nonscaled)
  X = scaleInput.fit_transform(X_nonscaled)
  X_train = X[:90]
  y_train = y[:90]
  X_valid = X[35:]
  y_valid = y[35:]
  stock_nnet = MLPClassifier(
    hidden_layer_sizes=(2), 
    activation='logistic', 
    solver='lbfgs', 
    random_state=1)
  stock_nnet.fit(X_train, y_train.ravel())
  y_pred = stock_nnet.predict(X_valid)
  accuracy_scores[ticker]= accuracy_score(y_valid, y_pred)
accuracy_scores

/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REAC

{'XLE': 0.5444444444444444,
 'XLF': 0.4666666666666667,
 'XLU': 0.5555555555555556,
 'XLI': 0.7111111111111111,
 'XLK': 0.6555555555555556,
 'XLV': 0.7333333333333333,
 'XLY': 0.5,
 'IYR': 0.6222222222222222,
 'AAPL': 0.6444444444444445,
 'XLB': 0.6111111111111112,
 'XLP': 0.6333333333333333,
 'SPY': 0.6222222222222222,
 '^DJI': 0.5888888888888889,
 'NDX': 0.6777777777777778}

In [7]:
stocks_df[:60]

,XLE,guardian_headline_mean,guardian_body_mean,Huff_headline_mean,Huff_body_mean,NYT_headline_mean,NYT_body_mean
0,0.0,-0.13,-0.01,-0.12,-0.06,-0.05,-0.03
1,1.0,-0.20,-0.12,-0.44,-0.39,-0.09,0.02
2,0.0,-0.08,-0.16,0.10,-0.09,-0.06,-0.00
3,1.0,-0.10,-0.24,-0.20,-0.14,-0.02,-0.05
4,1.0,-0.09,-0.09,-0.35,-0.30,-0.06,-0.05
5,1.0,-0.12,-0.12,0.01,-0.13,-0.08,0.05
6,0.0,-0.19,-0.07,-0.17,-0.35,0.01,-0.07
7,0.0,-0.09,0.08,0.27,-0.38,0.00,0.04
8,0.0,-0.20,-0.10,0.11,-0.03,0.03,0.05
9,1.0,-0.12,-0.02,0.08,0.16,-0.08,0.16


In [8]:
# separate out output neuron and input neurons
y_nonscaled = stocks_df[['XLE']]
X_nonscaled = stocks_df.drop(columns=['XLE'])
X_nonscaled.head()

,guardian_headline_mean,guardian_body_mean,Huff_headline_mean,Huff_body_mean,NYT_headline_mean,NYT_body_mean
0,-0.13,-0.01,-0.12,-0.06,-0.05,-0.03
1,-0.20,-0.12,-0.44,-0.39,-0.09,0.02
2,-0.08,-0.16,0.10,-0.09,-0.06,-0.00
3,-0.10,-0.24,-0.20,-0.14,-0.02,-0.05
4,-0.09,-0.09,-0.35,-0.30,-0.06,-0.05


In [22]:
# normalize the data into [0,1] range
# MinMax = (X - Xmin) / (Xmax - Xmin)
scaleOutput = MinMaxScaler()
scaleInput = MinMaxScaler()

# Why normalize the y variable? it can sometimes help, in particular, when
# the output activation function is bounded! (e.g., logistic remains within [0,1])

#y = scaleOutput.fit_transform(y_nonscaled)
y = scaleOutput.fit_transform(y_nonscaled)
X = scaleInput.fit_transform(X_nonscaled)

# partition data
#X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=1)
X_train = X[:90]
y_train = y[:90]
X_valid = X[35:]
y_valid = y[35:]

In [23]:
y

array([[0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],

In [24]:
# train neural network with single hidden layer and 2 nodes
# function: MLPRegressor
# https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html
# options:
# hidden_layer_sizes = (2, 4, ..., 1)  number of neurons in each hidden layer
# activation = 'logistic'
# solver = 'lbfgs'
stock_nnet = MLPClassifier(
    hidden_layer_sizes=(2), 
    activation='logistic', 
    solver='lbfgs', 
    random_state=1)
stock_nnet.fit(X_train, y_train.ravel())

MLPClassifier(activation='logistic', hidden_layer_sizes=2, random_state=1,
              solver='lbfgs')

In [25]:
y_pred = stock_nnet.predict(X_valid)
y_pred

array([1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.,
       1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1.,
       1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1.,
       1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1.,
       1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0.,
       0., 0., 0., 1., 1.])

In [26]:
y_valid

array([[1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],

In [27]:
accuracy_score(y_valid, y_pred)

0.5444444444444444

In [ ]:
# Inverse scaling using inverse_transform function
# Note: append the ravel() function to reformat the output of inverse_transform()
y_actual = scaleOutput.inverse_transform(y_valid).ravel()
y_pred = scaleOutput.inverse_transform([stock_nnet.predict(X_valid)]).ravel()
regressionSummary(y_pred, y_actual)
%matplotlib inline
#pd.DataFrame({'actual': y_actual, 'predicted': y_pred}).plot.scatter(x='actual', y='predicted', alpha=0.5)